In [1]:
import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup

In [2]:
url = 'https://pokemondb.net/pokedex/all'
html = requests.get(url).content

soup = BeautifulSoup(html, 'html.parser')
table = soup.find_all('div', class_='resp-scroll')
df = pd.read_html(str(table))[0]

df['Type'] = df['Type'].str.replace('  ', ' ')
df.drop('#', axis=1, inplace=True)
df.head()

,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,Charmander,Fire,309,39,52,43,60,50,65


In [3]:
url_dual_type = 'https://pokemondb.net/type/dual'
html_dual_type = requests.get(url_dual_type).content

soup_dual = BeautifulSoup(html_dual_type, 'html.parser')
table_type = soup_dual.find_all('div', class_='resp-scroll')[0]
dof = pd.read_html(str(table_type))[0]

dof = dof[dof['ATTACK → DEFENSE ↴'] != 'ATTACK → DEFENSE ↴']
dof.drop('PKMN', axis=1, inplace=True)
dof.columns = ['Type', 'Normal', 'Fire', 'Water', 'Electric', 'Grass', 'Ice', 'Fighting', 'Poison',
            'Ground', 'Flying', 'Psychic', 'Bug', 'Rock', 'Ghost', 'Dragon', 'Dark', 'Steel', 'Fairy']
dof.fillna(1, inplace=True)
dof.replace({'½':0.5, '¼':0.25}, inplace=True)
dof['Type'] = dof['Type'].str.replace('—', '')
dof['Type'] = dof['Type'].str.strip()
dof = dof.astype({col: 'float' for col in dof.columns[1:]})
dof.head()

,ATTACK → DEFENSE ↴,PKMN,Nor,Fir,Wat,Ele,Gra,Ice,Fig,Poi,Gro,Fly,Psy,Bug,Roc,Gho,Dra,Dar,Ste,Fai
0,Normal —,79,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1,Normal Fire,2,NaN,½,2,NaN,½,½,2,NaN,2,NaN,NaN,½,2,0,NaN,NaN,½,½
2,Normal Water,1,NaN,½,½,2,2,½,2,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,½,NaN
3,Normal Electric,2,NaN,NaN,NaN,½,NaN,NaN,2,NaN,2,½,NaN,NaN,NaN,0,NaN,NaN,½,NaN
4,Normal Grass,5,NaN,2,½,½,½,2,2,2,½,2,NaN,2,NaN,0,NaN,NaN,NaN,NaN


In [12]:
html_moves = requests.get('https://pokemondb.net/move/all').content

soup_moves = BeautifulSoup(html_moves, 'html.parser')
table_moves = soup_moves.find_all('div', class_='resp-scroll')
moves = pd.read_html(str(table_moves))[0]

moves['Power'].replace({'—':0}, inplace=True)
moves['Power'] = moves['Power'].astype('int')
moves.drop(['Cat.', 'Acc.', 'PP', 'Prob. (%)'], axis=1, inplace=True)
moves.head()

,Name,Type,Power,Effect
0,"10,000,000 Volt Thunderbolt",Electric,195,Pikachu-exclusive Z-Move. High critical hit ra...
1,Absorb,Grass,20,User recovers half the HP inflicted on opponent.
2,Accelerock,Rock,40,User attacks first.
3,Acid,Poison,40,May lower opponent's Special Defense.
4,Acid Armor,Poison,0,Sharply raises user's Defense.


In [ ]:
class AtkPokemon:
    url_pokemon = 'https://pokemondb.net/pokedex/all'
    url_moves = 'https://pokemondb.net/move/all'

    def __init__(self, name, level=50):
        self.name = name
        self.stats = self.get_stats()
        self.type = self.stats[self.stats['Name'] == name]['Type'].iloc[0]
        self.hp = int(self.stats[self.stats['Name'] == name]['HP'].iloc[0])
        self.attack =int(self.stats[self.stats['Name'] == name]['Attack'].iloc[0])
        self.defense = self.stats[self.stats['Name'] == name]['Defense']
        self.sp_attack = self.stats[self.stats['Name'] == name]['Sp. Atk']
        self.sp_defense = self.stats[self.stats['Name'] == name]['Sp. Def']
        self.speed = self.stats[self.stats['Name'] == name]['Speed']
        self.level = level

    @classmethod
    def get_stats(cls):
        html = requests.get(cls.url_pokemon).content

        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('div', class_='resp-scroll')
        stats = pd.read_html(str(table))[0]

        stats['Type'] = stats['Type'].str.replace('  ', ' ')
        return stats

class DefPokemon:
    url_pokemon = 'https://pokemondb.net/pokedex/all'
    url_moves = 'https://pokemondb.net/move/all'

    def __init__(self, name):
        self.name = name
        self.stats = self.get_stats()
        self.type = self.stats[self.stats['Name'] == name]['Type'].iloc[0]
        self.hp = int(self.stats[self.stats['Name'] == name]['HP'].iloc[0])
        self.attack = self.stats[self.stats['Name'] == name]['Attack']
        self.defense = int(self.stats[self.stats['Name'] == name]['Defense'].iloc[0])
        self.sp_attack = self.stats[self.stats['Name'] == name]['Sp. Atk']
        self.sp_defense = self.stats[self.stats['Name'] == name]['Sp. Def']
        self.speed = self.stats[self.stats['Name'] == name]['Speed']

    @classmethod
    def get_stats(cls):
        html = requests.get(cls.url_pokemon).content

        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('div', class_='resp-scroll')
        stats = pd.read_html(str(table))[0]

        stats['Type'] = stats['Type'].str.replace('  ', ' ')
        return stats
    
class DamageCalculator:
    url_pokemon = 'https://pokemondb.net/pokedex/all'
    url_moves = 'https://pokemondb.net/move/all'
    
    def __init__(self, atk_pokemon, def_pokemon, move):
        self.atk_pokemon = AtkPokemon(atk_pokemon)
        self.def_pokemon = DefPokemon(def_pokemon)
        self.moves = self.get_moves()
        self.move = self.moves[self.moves['Name'] == move]
        self.move_power = int(self.move['Power'].iloc[0])
        self.move_type = self.move['Type'].iloc[0]

    @classmethod
    def get_moves(cls):
        html = requests.get(cls.url_moves).content

        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('div', class_='resp-scroll')
        moves = pd.read_html(str(table))[0]

        moves['Power'].replace({'—':0}, inplace=True)
        moves['Power'] = moves['Power'].astype('int')
        return moves
    
    def type_effectiveness(self):
        url_dual_type = 'https://pokemondb.net/type/dual'
        html_dual_type = requests.get(url_dual_type).content

        soup_dual = BeautifulSoup(html_dual_type, 'html.parser')
        table_type = soup_dual.find_all('div', class_='resp-scroll')[0]
        df = pd.read_html(str(table_type))[0]

        df = df[df['ATTACK → DEFENSE ↴'] != 'ATTACK → DEFENSE ↴']
        df.drop('PKMN', axis=1, inplace=True)
        df.columns = ['Type', 'Normal', 'Fire', 'Water', 'Electric', 'Grass', 'Ice', 'Fighting', 'Poison',
                    'Ground', 'Flying', 'Psychic', 'Bug', 'Rock', 'Ghost', 'Dragon', 'Dark', 'Steel', 'Fairy']
        df.fillna(1, inplace=True)
        df.replace({'½':0.5, '¼':0.25}, inplace=True)
        df['Type'] = df['Type'].str.replace('—', '')
        df['Type'] = df['Type'].str.strip()
        df = df.astype({col: 'float' for col in df.columns[1:]})
        
        self.type_multiplier = df[df['Type'] == self.def_pokemon.type]
        self.type_multiplier = self.type_multiplier[self.move_type].iloc[0]
        return self.type_multiplier
        
    def result(self, target=1):
        if target == 1:
            pass
        else:
            target = 0.75
        damage = ((((((2*self.atk_pokemon.level)/5)+2) * self.move_power*self.atk_pokemon.attack/self.def_pokemon.defense)/50) + 2)
        damage_1 = damage * target * self.type_effectiveness() 
        damage_0 = damage * target * self.type_effectiveness() * 0.85

        return print(f'Range de dano: [{(damage_0*100 / self.def_pokemon.hp):.2f} % - {(damage_1*100 / self.def_pokemon.hp):.2f} %]')